In [3]:
pip install pulp

  Obtaining dependency information for pulp from https://files.pythonhosted.org/packages/09/d7/57e71e11108203039c895643368c0d1a99fe719a6a80184edf240c33d25f/PuLP-2.8.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 13.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger
import pandas as pd

In [ ]:
# Load the data from an Excel file
file_path = 'table5.xlsx'
data = pd.read_excel(file_path)

# Define constants
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
airports = ["A", "B", "C"]
pairs = [(i, j) for i in airports for j in airports if i != j]
transport_cost = {('A', 'B'): 7, ('B', 'A'): 7, ('A', 'C'): 3,
                  ('C', 'A'): 3, ('B', 'C'): 6, ('C', 'B'): 6}
idle_cost_per_goods = 10
total_planes = 1200

# Initialize the problem
model = LpProblem("Airplane_and_Goods_Transportation", LpMinimize)

# Define variables
init = LpVariable.dicts("initial_planes", airports, lowBound=0, cat=LpInteger)
x = LpVariable.dicts("cargo_planes", (days, pairs), lowBound=0, cat=LpInteger)
y = LpVariable.dicts("excess_planes", (days, pairs), lowBound=0, cat=LpInteger)
z = LpVariable.dicts("idle_goods", (days, pairs), lowBound=0, cat=LpInteger)

# Initialize a dictionary to keep track of retained planes
retained_planes = {day: {airport: 0 for airport in airports} for day in days}

# Objective function
model += (
    lpSum(transport_cost[pair] * y[day][pair] for day in days for pair in pairs)+
    lpSum(idle_cost_per_goods * z[day][pair] for day in days for pair in pairs)
)

# Constraints
# Total initial number of planes constraint
model += lpSum(init[airport] for airport in airports) == total_planes

# Cumulative goods demand satisfaction constraint
for pair in pairs:
    model += lpSum(x[day][pair] for day in days) >= \
        lpSum(data.loc[data['Origin-Destination'] == f"{pair[0]}-{pair[1]}", day]
              for day in days)

# Calculations for excess planes and idle goods
for day in days:
    for i, j in pairs:
        required_daily = data.loc[data['Origin-Destination'] == \
                                  f"{i}-{j}", day].values[0]
        model += y[day][(i, j)] >= x[day][(i, j)] - required_daily
        model += z[day][(i, j)] >= required_daily - x[day][(i, j)]

# Weekend airplane restoration condition  
for airport in airports:
    model += (
        init[airport] + lpSum(x[day][(k, airport)] + y[day][(k, airport)]
                              - x[day][(airport, k)] - y[day][(airport, k)]
                              for k in airports if k != airport for day in days) == init[airport]
    )

# Initialize the retained planes dictionary
retained_planes = {airport: {day: init[airport] for day in days} \
                   for airport in airports}

# Update the number of retained planes
for day_index, day in enumerate(days):
    if day_index > 0: # Starting from the second day
        prev_day = days[day_index - 1]
        for airport in airports:
            # Calculate arriving and departing planes  
            arrivals = lpSum(x[prev_day][(k, airport)] + \
                             y[prev_day][(k, airport)] for k in airports if k != airport)
            departures = lpSum(x[prev_day][(airport, k)] + \
                               y[prev_day][(airport, k)] for k in airports if k != airport) 
            retained_planes[airport][day]=retained_planes[airport][prev_day] + \
                arrivals - departures

# Add daily plane balance constraints
for day in days:
    for airport in airports:
        model += lpSum(x[day][(airport, k)] + y[day][(airport, k)]  
                       for k in airports if k != airport) <= \
            retained_planes[airport][day]

model.solve()

# Initialize DataFrame structures
movement_columns = ['Day', 'Route', 'Cargo Planes', 'Empty Planes']  
movement_data = []
airport_status_columns = ['Day','Airport','Initial Planes',\
                          'Planes Departed','Planes Arrived']
airport_status_data = []

# Populate data into DataFrame
for day in days:
    for i, j in pairs:
        movement_data.append({'Day': day, 'Route': f"{i}-{j}",  
                              'Cargo Planes': x[day][(i, j)].varValue if x[day][(i, j)].varValue \
                                  is not None else 0,
                              'Empty Planes': y[day][(i, j)].varValue if y[day][(i, j)].varValue \
                                  is not None else 0                            
                             })

    for airport in airports:
        initial_planes = init[airport].varValue if day == "Monday" else None
        planes_departed = sum(x[day][(airport, k)].varValue + \
                              y[day][(airport, k)].varValue for k in airports if k != airport)
        planes_arrived = sum(x[day][(k, airport)].varValue + \
                             y[day][(k, airport)].varValue for k in airports if k != airport)
        airport_status_data.append({
            'Day': day,
            'Airport': airport,
            'Initial Planes': initial_planes,
            'Planes Departed': planes_departed,
            'Planes Arrived': planes_arrived
        })

# Create DataFrames
movement_df=pd.DataFrame(movement_data,columns=movement_columns)
airport_status_df=pd.DataFrame(airport_status_data,\
                               columns=airport_status_columns)

# Export DataFrames to CSV
movement_df.to_csv('res1.csv')  
airport_status_df.to_csv('res2.csv')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yulongquan/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j2/kn24rgbs61zdgt6q877h9x0c0000gn/T/18d5f139ccf94cdaa53b1fb5a6412efe-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/j2/kn24rgbs61zdgt6q877h9x0c0000gn/T/18d5f139ccf94cdaa53b1fb5a6412efe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 90 COLUMNS
At line 928 RHS
At line 1014 BOUNDS
At line 1108 ENDATA
Problem MODEL has 85 rows, 93 columns and 588 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10637.5 - 0.00 seconds
Cgl0003I 0 fixed, 118 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 100 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 64 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 32 t

Cbc0010I After 99000 nodes, 88 on tree, 10639 best solution, best possible 10637.5 (51.98 seconds)
Cbc0010I After 100000 nodes, 98 on tree, 10639 best solution, best possible 10637.5 (52.48 seconds)
Cbc0010I After 101000 nodes, 91 on tree, 10639 best solution, best possible 10637.5 (52.89 seconds)
Cbc0010I After 102000 nodes, 49 on tree, 10639 best solution, best possible 10637.5 (53.32 seconds)
Cbc0010I After 103000 nodes, 38 on tree, 10639 best solution, best possible 10637.5 (53.71 seconds)
Cbc0010I After 104000 nodes, 87 on tree, 10639 best solution, best possible 10637.5 (54.09 seconds)
Cbc0010I After 105000 nodes, 86 on tree, 10639 best solution, best possible 10637.5 (54.47 seconds)
Cbc0010I After 106000 nodes, 186 on tree, 10639 best solution, best possible 10637.5 (54.85 seconds)
Cbc0010I After 107000 nodes, 263 on tree, 10639 best solution, best possible 10637.5 (55.15 seconds)
Cbc0038I Full problem 51 rows 59 columns, reduced to 38 rows 39 columns
Cbc0010I After 108000 nodes

Cbc0010I After 258000 nodes, 1354 on tree, 10639 best solution, best possible 10637.5 (105.13 seconds)
Cbc0010I After 259000 nodes, 1012 on tree, 10639 best solution, best possible 10637.5 (105.48 seconds)
Cbc0010I After 260000 nodes, 951 on tree, 10639 best solution, best possible 10637.5 (105.93 seconds)
Cbc0010I After 261000 nodes, 898 on tree, 10639 best solution, best possible 10637.5 (106.23 seconds)
Cbc0010I After 262000 nodes, 961 on tree, 10639 best solution, best possible 10637.5 (106.49 seconds)
Cbc0010I After 263000 nodes, 965 on tree, 10639 best solution, best possible 10637.5 (106.78 seconds)
Cbc0010I After 264000 nodes, 925 on tree, 10639 best solution, best possible 10637.5 (107.09 seconds)
Cbc0010I After 265000 nodes, 928 on tree, 10639 best solution, best possible 10637.5 (107.44 seconds)
Cbc0010I After 266000 nodes, 1132 on tree, 10639 best solution, best possible 10637.5 (107.91 seconds)
Cbc0010I After 267000 nodes, 1053 on tree, 10639 best solution, best possible 1

Cbc0010I After 415000 nodes, 228 on tree, 10639 best solution, best possible 10637.5 (158.39 seconds)
Cbc0010I After 416000 nodes, 183 on tree, 10639 best solution, best possible 10637.5 (158.66 seconds)
Cbc0010I After 417000 nodes, 149 on tree, 10639 best solution, best possible 10637.5 (159.09 seconds)
Cbc0010I After 418000 nodes, 165 on tree, 10639 best solution, best possible 10637.5 (159.35 seconds)
Cbc0010I After 419000 nodes, 152 on tree, 10639 best solution, best possible 10637.5 (159.60 seconds)
Cbc0010I After 420000 nodes, 157 on tree, 10639 best solution, best possible 10637.5 (159.88 seconds)
Cbc0010I After 421000 nodes, 139 on tree, 10639 best solution, best possible 10637.5 (160.28 seconds)
Cbc0010I After 422000 nodes, 212 on tree, 10639 best solution, best possible 10637.5 (160.55 seconds)
Cbc0010I After 423000 nodes, 243 on tree, 10639 best solution, best possible 10637.5 (160.81 seconds)
Cbc0010I After 424000 nodes, 230 on tree, 10639 best solution, best possible 10637

Cbc0010I After 575000 nodes, 205 on tree, 10639 best solution, best possible 10637.5 (203.09 seconds)
Cbc0010I After 576000 nodes, 299 on tree, 10639 best solution, best possible 10637.5 (203.43 seconds)
Cbc0010I After 577000 nodes, 310 on tree, 10639 best solution, best possible 10637.5 (203.72 seconds)
Cbc0010I After 578000 nodes, 294 on tree, 10639 best solution, best possible 10637.5 (204.01 seconds)
Cbc0010I After 579000 nodes, 238 on tree, 10639 best solution, best possible 10637.5 (204.29 seconds)
Cbc0010I After 580000 nodes, 227 on tree, 10639 best solution, best possible 10637.5 (204.58 seconds)
Cbc0010I After 581000 nodes, 250 on tree, 10639 best solution, best possible 10637.5 (204.87 seconds)
Cbc0010I After 582000 nodes, 305 on tree, 10639 best solution, best possible 10637.5 (205.18 seconds)
Cbc0010I After 583000 nodes, 339 on tree, 10639 best solution, best possible 10637.5 (205.43 seconds)
Cbc0010I After 584000 nodes, 350 on tree, 10639 best solution, best possible 10637

Cbc0010I After 735000 nodes, 250 on tree, 10639 best solution, best possible 10637.5 (258.11 seconds)
Cbc0010I After 736000 nodes, 198 on tree, 10639 best solution, best possible 10637.5 (258.52 seconds)
Cbc0010I After 737000 nodes, 206 on tree, 10639 best solution, best possible 10637.5 (258.88 seconds)
Cbc0010I After 738000 nodes, 236 on tree, 10639 best solution, best possible 10637.5 (259.20 seconds)
Cbc0010I After 739000 nodes, 190 on tree, 10639 best solution, best possible 10637.5 (259.61 seconds)
Cbc0010I After 740000 nodes, 205 on tree, 10639 best solution, best possible 10637.5 (259.92 seconds)
Cbc0010I After 741000 nodes, 306 on tree, 10639 best solution, best possible 10637.5 (260.32 seconds)
Cbc0010I After 742000 nodes, 282 on tree, 10639 best solution, best possible 10637.5 (260.68 seconds)
Cbc0010I After 743000 nodes, 274 on tree, 10639 best solution, best possible 10637.5 (261.07 seconds)
Cbc0010I After 744000 nodes, 223 on tree, 10639 best solution, best possible 10637